In [128]:
# Importing Pandas, a data processing and CSV file I/O libraries
import os
import pandas as pd # type: ignore
import numpy as np # type: ignore
import seaborn as sns # Seaborn is a Python data visualization library based on matplotlib. # type: ignore
%matplotlib inline
import pathlib
from datetime import datetime
import math
import sys




import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

import xgboost as xg
from sklearn.ensemble import RandomForestRegressor

%matplotlib inline

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)




In [129]:
df_pre = pd.read_csv('/Users/yashwanthkaruparthi/Developer/energy_demand/research/data/8-feat-eng/feat-lag.csv')

df_orig = pd.read_csv('/Users/yashwanthkaruparthi/Developer/energy_demand/research/data/7-usable/1-daily_data.csv')

In [130]:
df_pre.columns

Index(['Unnamed: 0', 'temp_min', 'temp_max', 'temp_mean', 'temp_median', 'temp_max_hour', 'temp_min_hour', 'dew_point_temp_min', 'dew_point_temp_max', 'dew_point_temp_mean', 'dew_point_temp_median', 'dew_point_temp_max_hour', 'dew_point_temp_min_hour', 'rel_hum_min', 'rel_hum_max', 'rel_hum_mean', 'rel_hum_median', 'rel_hum_max_hour', 'rel_hum_min_hour', 'visibility_min', 'visibility_max', 'visibility_mean', 'visibility_median', 'visibility_max_hour', 'visibility_min_hour', 'press_min', 'press_max', 'press_mean', 'press_median', 'press_max_hour', 'press_min_hour', 'hmdxx_min', 'hmdxx_max', 'hmdxx_mean', 'hmdxx_median', 'hmdxx_max_hour', 'hmdxx_min_hour', 'sun_set', 'daily_demand', 'day_of_week', 'week_of_year', 'year', 'day', 'month', 'demand_rolling_mean_3', 'demand_rolling_std_7', 'demand_lag_1', 'demand_lag_2', 'demand_lag_3', 'demand_lag_4', 'demand_lag_5', 'demand_lag_6', 'demand_lag_7'], dtype='object')

In [131]:
df_orig.columns

Index(['Unnamed: 0', 'date', 'temp', 'dew_point_temp', 'rel_hum', 'wind_speed', 'visibility', 'press', 'wind_chill', 'hourly_demand', 'hmdxx', 'temp_min', 'temp_max', 'temp_mean', 'temp_median', 'dew_point_temp_min', 'dew_point_temp_max', 'dew_point_temp_mean', 'dew_point_temp_median', 'rel_hum_min', 'rel_hum_max', 'rel_hum_mean', 'rel_hum_median', 'visibility_min', 'visibility_max', 'visibility_mean', 'visibility_median', 'press_min', 'press_max', 'press_mean', 'press_median', 'hmdxx_min', 'hmdxx_max', 'hmdxx_mean', 'hmdxx_median'], dtype='object')

In [132]:
df_orig['date'] = pd.to_datetime(df_orig['date'])

In [133]:
df_orig.set_index('date', inplace=True)

In [134]:
# df_orig['hour_of_day'] = df_orig.index.hour
df_orig['year'] = df_orig.index.year
df_orig['month'] = df_orig.index.month
df_orig['day_of_week'] = df_orig.index.dayofweek
df_orig['day_of_year'] = df_orig.index.dayofyear
# df_orig['week_of_year'] = df_orig.index.weekofyear
df_orig['week_of_year'] = df_orig.index.isocalendar().week

In [135]:
df_orig.columns

Index(['Unnamed: 0', 'temp', 'dew_point_temp', 'rel_hum', 'wind_speed', 'visibility', 'press', 'wind_chill', 'hourly_demand', 'hmdxx', 'temp_min', 'temp_max', 'temp_mean', 'temp_median', 'dew_point_temp_min', 'dew_point_temp_max', 'dew_point_temp_mean', 'dew_point_temp_median', 'rel_hum_min', 'rel_hum_max', 'rel_hum_mean', 'rel_hum_median', 'visibility_min', 'visibility_max', 'visibility_mean', 'visibility_median', 'press_min', 'press_max', 'press_mean', 'press_median', 'hmdxx_min', 'hmdxx_max', 'hmdxx_mean', 'hmdxx_median', 'year', 'month', 'day_of_week', 'day_of_year', 'week_of_year'], dtype='object')

In [136]:
df_orig.head()

,Unnamed: 0,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,wind_chill,hourly_demand,hmdxx,temp_min,temp_max,temp_mean,temp_median,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,visibility_min,visibility_max,visibility_mean,visibility_median,press_min,press_max,press_mean,press_median,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median,year,month,day_of_week,day_of_year,week_of_year
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1994-01-01,0,0.575000,-1.387500,87.000000,20.333333,17.841667,99.075833,-5.888889,14655.208333,-1.884900,-1.8,2.8,0.575000,0.65,-4.8,1.1,-1.387500,-0.65,73.0,99.0,87.000000,86.5,0.8,40.2,17.841667,16.1,98.51,99.91,99.075833,98.840,-4.976663,0.686009,-1.884900,-1.513575,1994,1,5,1,52
1994-01-02,1,-8.704167,-12.445833,74.416667,17.875000,28.387500,99.920000,-16.681818,15783.333333,-12.827363,-14.3,1.7,-8.704167,-9.90,-20.3,0.5,-12.445833,-13.90,60.0,92.0,74.416667,75.5,6.4,40.2,28.387500,25.0,98.77,100.51,99.920000,100.125,-19.165499,-0.338394,-12.827363,-14.393998,1994,1,6,2,52
1994-01-03,2,-12.612500,-15.820833,77.250000,17.666667,10.404167,100.057083,-20.583333,18992.708333,-17.143932,-16.3,-10.3,-12.612500,-12.00,-23.2,-12.6,-15.820833,-15.00,55.0,91.0,77.250000,80.5,1.2,25.0,10.404167,8.0,99.41,100.47,100.057083,100.155,-21.317386,-14.555717,-17.143932,-16.423198,1994,1,0,3,1
1994-01-04,3,-9.833333,-13.570833,74.416667,25.791667,24.879167,98.675417,-18.541667,19080.708333,-14.183286,-11.5,-7.4,-9.833333,-9.95,-15.0,-11.5,-13.570833,-13.70,57.0,84.0,74.416667,76.0,9.7,40.2,24.879167,24.1,98.27,99.34,98.675417,98.610,-15.874948,-11.706660,-14.183286,-14.310177,1994,1,1,4,1
1994-01-05,4,-11.866667,-16.154167,70.791667,22.875000,27.033333,99.352083,-20.521739,19183.208333,-16.407695,-17.5,-7.2,-11.866667,-11.75,-20.8,-10.7,-16.154167,-17.00,56.0,84.0,70.791667,74.0,1.0,40.2,27.033333,25.0,98.44,100.37,99.352083,99.375,-22.394182,-11.256395,-16.407695,-16.464188,1994,1,2,5,1


In [137]:
df_pre.head()

,Unnamed: 0,temp_min,temp_max,temp_mean,temp_median,temp_max_hour,temp_min_hour,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,dew_point_temp_max_hour,dew_point_temp_min_hour,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,rel_hum_max_hour,rel_hum_min_hour,visibility_min,visibility_max,visibility_mean,visibility_median,visibility_max_hour,visibility_min_hour,press_min,press_max,press_mean,press_median,press_max_hour,press_min_hour,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median,hmdxx_max_hour,hmdxx_min_hour,sun_set,daily_demand,day_of_week,week_of_year,year,day,month,demand_rolling_mean_3,demand_rolling_std_7,demand_lag_1,demand_lag_2,demand_lag_3,demand_lag_4,demand_lag_5,demand_lag_6,demand_lag_7
0,0,-1.8,2.8,0.575000,0.65,12.0,5.0,-4.8,1.1,-1.387500,-0.65,19.0,7.0,73.0,99.0,87.000000,86.5,14.0,9.0,0.8,40.2,17.841667,16.1,11.0,14.0,98.51,99.91,99.075833,98.840,0.0,16.0,-4.976663,0.686009,-1.884900,-1.513575,13.0,7.0,17.0,14152.0,5.0,52.0,1994,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,-14.3,1.7,-8.704167,-9.90,0.0,23.0,-20.3,0.5,-12.445833,-13.90,0.0,23.0,60.0,92.0,74.416667,75.5,0.0,23.0,6.4,40.2,28.387500,25.0,9.0,6.0,98.77,100.51,99.920000,100.125,18.0,0.0,-19.165499,-0.338394,-12.827363,-14.393998,0.0,23.0,17.0,16527.0,6.0,52.0,1994,2,1,NaN,NaN,14152.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,-16.3,-10.3,-12.612500,-12.00,17.0,2.0,-23.2,-12.6,-15.820833,-15.00,17.0,2.0,55.0,91.0,77.250000,80.5,9.0,2.0,1.2,25.0,10.404167,8.0,0.0,9.0,99.41,100.47,100.057083,100.155,0.0,23.0,-21.317386,-14.555717,-17.143932,-16.423198,17.0,2.0,17.0,17550.0,0.0,1.0,1994,3,1,16076.333333,NaN,16527.0,14152.0,NaN,NaN,NaN,NaN,NaN
3,3,-11.5,-7.4,-9.833333,-9.95,15.0,8.0,-15.0,-11.5,-13.570833,-13.70,13.0,0.0,57.0,84.0,74.416667,76.0,10.0,16.0,9.7,40.2,24.879167,24.1,13.0,10.0,98.27,99.34,98.675417,98.610,0.0,14.0,-15.874948,-11.706660,-14.183286,-14.310177,15.0,8.0,17.0,17395.0,1.0,1.0,1994,4,1,17157.333333,NaN,17550.0,16527.0,14152.0,NaN,NaN,NaN,NaN
4,4,-17.5,-7.2,-11.866667,-11.75,5.0,22.0,-20.8,-10.7,-16.154167,-17.00,4.0,22.0,56.0,84.0,70.791667,74.0,4.0,14.0,1.0,40.2,27.033333,25.0,9.0,7.0,98.44,100.37,99.352083,99.375,22.0,3.0,-22.394182,-11.256395,-16.407695,-16.464188,5.0,22.0,17.0,18485.0,2.0,1.0,1994,5,1,17810.000000,NaN,17395.0,17550.0,16527.0,14152.0,NaN,NaN,NaN


In [138]:
df_orig.drop(columns=['wind_speed', 'wind_chill', 'temp_min', 'temp_max', 'temp_mean', 'temp_median', 'dew_point_temp_min', 'dew_point_temp_max', 'dew_point_temp_mean', 'dew_point_temp_median', 'rel_hum_min', 'rel_hum_max', 'rel_hum_mean', 'rel_hum_median', 'visibility_min', 'visibility_max', 'visibility_mean', 'visibility_median', 'press_min', 'press_max', 'press_mean', 'press_median', 'hmdxx_min', 'hmdxx_max', 'hmdxx_mean', 'hmdxx_median'], inplace = True)

In [139]:
df_orig.head()

,Unnamed: 0,temp,dew_point_temp,rel_hum,visibility,press,hourly_demand,hmdxx,year,month,day_of_week,day_of_year,week_of_year
date,,,,,,,,,,,,,
1994-01-01,0,0.575000,-1.387500,87.000000,17.841667,99.075833,14655.208333,-1.884900,1994,1,5,1,52
1994-01-02,1,-8.704167,-12.445833,74.416667,28.387500,99.920000,15783.333333,-12.827363,1994,1,6,2,52
1994-01-03,2,-12.612500,-15.820833,77.250000,10.404167,100.057083,18992.708333,-17.143932,1994,1,0,3,1
1994-01-04,3,-9.833333,-13.570833,74.416667,24.879167,98.675417,19080.708333,-14.183286,1994,1,1,4,1
1994-01-05,4,-11.866667,-16.154167,70.791667,27.033333,99.352083,19183.208333,-16.407695,1994,1,2,5,1


In [140]:
df_pre.head()

,Unnamed: 0,temp_min,temp_max,temp_mean,temp_median,temp_max_hour,temp_min_hour,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,dew_point_temp_max_hour,dew_point_temp_min_hour,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,rel_hum_max_hour,rel_hum_min_hour,visibility_min,visibility_max,visibility_mean,visibility_median,visibility_max_hour,visibility_min_hour,press_min,press_max,press_mean,press_median,press_max_hour,press_min_hour,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median,hmdxx_max_hour,hmdxx_min_hour,sun_set,daily_demand,day_of_week,week_of_year,year,day,month,demand_rolling_mean_3,demand_rolling_std_7,demand_lag_1,demand_lag_2,demand_lag_3,demand_lag_4,demand_lag_5,demand_lag_6,demand_lag_7
0,0,-1.8,2.8,0.575000,0.65,12.0,5.0,-4.8,1.1,-1.387500,-0.65,19.0,7.0,73.0,99.0,87.000000,86.5,14.0,9.0,0.8,40.2,17.841667,16.1,11.0,14.0,98.51,99.91,99.075833,98.840,0.0,16.0,-4.976663,0.686009,-1.884900,-1.513575,13.0,7.0,17.0,14152.0,5.0,52.0,1994,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,-14.3,1.7,-8.704167,-9.90,0.0,23.0,-20.3,0.5,-12.445833,-13.90,0.0,23.0,60.0,92.0,74.416667,75.5,0.0,23.0,6.4,40.2,28.387500,25.0,9.0,6.0,98.77,100.51,99.920000,100.125,18.0,0.0,-19.165499,-0.338394,-12.827363,-14.393998,0.0,23.0,17.0,16527.0,6.0,52.0,1994,2,1,NaN,NaN,14152.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,-16.3,-10.3,-12.612500,-12.00,17.0,2.0,-23.2,-12.6,-15.820833,-15.00,17.0,2.0,55.0,91.0,77.250000,80.5,9.0,2.0,1.2,25.0,10.404167,8.0,0.0,9.0,99.41,100.47,100.057083,100.155,0.0,23.0,-21.317386,-14.555717,-17.143932,-16.423198,17.0,2.0,17.0,17550.0,0.0,1.0,1994,3,1,16076.333333,NaN,16527.0,14152.0,NaN,NaN,NaN,NaN,NaN
3,3,-11.5,-7.4,-9.833333,-9.95,15.0,8.0,-15.0,-11.5,-13.570833,-13.70,13.0,0.0,57.0,84.0,74.416667,76.0,10.0,16.0,9.7,40.2,24.879167,24.1,13.0,10.0,98.27,99.34,98.675417,98.610,0.0,14.0,-15.874948,-11.706660,-14.183286,-14.310177,15.0,8.0,17.0,17395.0,1.0,1.0,1994,4,1,17157.333333,NaN,17550.0,16527.0,14152.0,NaN,NaN,NaN,NaN
4,4,-17.5,-7.2,-11.866667,-11.75,5.0,22.0,-20.8,-10.7,-16.154167,-17.00,4.0,22.0,56.0,84.0,70.791667,74.0,4.0,14.0,1.0,40.2,27.033333,25.0,9.0,7.0,98.44,100.37,99.352083,99.375,22.0,3.0,-22.394182,-11.256395,-16.407695,-16.464188,5.0,22.0,17.0,18485.0,2.0,1.0,1994,5,1,17810.000000,NaN,17395.0,17550.0,16527.0,14152.0,NaN,NaN,NaN


In [141]:
df_orig.rename(columns = {'hourly_demand': 'daily_demand'}, inplace = True)

In [142]:
# features = ['sun']

# for feature in features:
#     min_hours = df_orig.loc[df_orig.groupby(pd.Grouper(freq='D')).idxmin().loc[:, feature]]['hour_of_day'].values
#     df_orig[feature + '_set'] = np.repeat(min_hours, 24)

In [143]:
df_orig.columns

Index(['Unnamed: 0', 'temp', 'dew_point_temp', 'rel_hum', 'visibility', 'press', 'daily_demand', 'hmdxx', 'year', 'month', 'day_of_week', 'day_of_year', 'week_of_year'], dtype='object')

In [144]:
df_pre['date'] = pd.to_datetime(df_pre[['year', 'month', 'day']])

In [145]:
df_pre.head()

,Unnamed: 0,temp_min,temp_max,temp_mean,temp_median,temp_max_hour,temp_min_hour,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,dew_point_temp_max_hour,dew_point_temp_min_hour,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,rel_hum_max_hour,rel_hum_min_hour,visibility_min,visibility_max,visibility_mean,visibility_median,visibility_max_hour,visibility_min_hour,press_min,press_max,press_mean,press_median,press_max_hour,press_min_hour,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median,hmdxx_max_hour,hmdxx_min_hour,sun_set,daily_demand,day_of_week,week_of_year,year,day,month,demand_rolling_mean_3,demand_rolling_std_7,demand_lag_1,demand_lag_2,demand_lag_3,demand_lag_4,demand_lag_5,demand_lag_6,demand_lag_7,date
0,0,-1.8,2.8,0.575000,0.65,12.0,5.0,-4.8,1.1,-1.387500,-0.65,19.0,7.0,73.0,99.0,87.000000,86.5,14.0,9.0,0.8,40.2,17.841667,16.1,11.0,14.0,98.51,99.91,99.075833,98.840,0.0,16.0,-4.976663,0.686009,-1.884900,-1.513575,13.0,7.0,17.0,14152.0,5.0,52.0,1994,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01
1,1,-14.3,1.7,-8.704167,-9.90,0.0,23.0,-20.3,0.5,-12.445833,-13.90,0.0,23.0,60.0,92.0,74.416667,75.5,0.0,23.0,6.4,40.2,28.387500,25.0,9.0,6.0,98.77,100.51,99.920000,100.125,18.0,0.0,-19.165499,-0.338394,-12.827363,-14.393998,0.0,23.0,17.0,16527.0,6.0,52.0,1994,2,1,NaN,NaN,14152.0,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-02
2,2,-16.3,-10.3,-12.612500,-12.00,17.0,2.0,-23.2,-12.6,-15.820833,-15.00,17.0,2.0,55.0,91.0,77.250000,80.5,9.0,2.0,1.2,25.0,10.404167,8.0,0.0,9.0,99.41,100.47,100.057083,100.155,0.0,23.0,-21.317386,-14.555717,-17.143932,-16.423198,17.0,2.0,17.0,17550.0,0.0,1.0,1994,3,1,16076.333333,NaN,16527.0,14152.0,NaN,NaN,NaN,NaN,NaN,1994-01-03
3,3,-11.5,-7.4,-9.833333,-9.95,15.0,8.0,-15.0,-11.5,-13.570833,-13.70,13.0,0.0,57.0,84.0,74.416667,76.0,10.0,16.0,9.7,40.2,24.879167,24.1,13.0,10.0,98.27,99.34,98.675417,98.610,0.0,14.0,-15.874948,-11.706660,-14.183286,-14.310177,15.0,8.0,17.0,17395.0,1.0,1.0,1994,4,1,17157.333333,NaN,17550.0,16527.0,14152.0,NaN,NaN,NaN,NaN,1994-01-04
4,4,-17.5,-7.2,-11.866667,-11.75,5.0,22.0,-20.8,-10.7,-16.154167,-17.00,4.0,22.0,56.0,84.0,70.791667,74.0,4.0,14.0,1.0,40.2,27.033333,25.0,9.0,7.0,98.44,100.37,99.352083,99.375,22.0,3.0,-22.394182,-11.256395,-16.407695,-16.464188,5.0,22.0,17.0,18485.0,2.0,1.0,1994,5,1,17810.000000,NaN,17395.0,17550.0,16527.0,14152.0,NaN,NaN,NaN,1994-01-05


In [146]:
df_orig = df_orig.reset_index()

In [147]:
df_orig.head()

,date,Unnamed: 0,temp,dew_point_temp,rel_hum,visibility,press,daily_demand,hmdxx,year,month,day_of_week,day_of_year,week_of_year
0,1994-01-01,0,0.575000,-1.387500,87.000000,17.841667,99.075833,14655.208333,-1.884900,1994,1,5,1,52
1,1994-01-02,1,-8.704167,-12.445833,74.416667,28.387500,99.920000,15783.333333,-12.827363,1994,1,6,2,52
2,1994-01-03,2,-12.612500,-15.820833,77.250000,10.404167,100.057083,18992.708333,-17.143932,1994,1,0,3,1
3,1994-01-04,3,-9.833333,-13.570833,74.416667,24.879167,98.675417,19080.708333,-14.183286,1994,1,1,4,1
4,1994-01-05,4,-11.866667,-16.154167,70.791667,27.033333,99.352083,19183.208333,-16.407695,1994,1,2,5,1


In [148]:
df_orig = pd.merge(df_orig, df_pre[['date', 'sun_set']], on=pd.to_datetime(df_orig['date']), how='left')

In [149]:
df_orig.head()

,key_0,date_x,Unnamed: 0,temp,dew_point_temp,rel_hum,visibility,press,daily_demand,hmdxx,year,month,day_of_week,day_of_year,week_of_year,date_y,sun_set
0,1994-01-01,1994-01-01,0,0.575000,-1.387500,87.000000,17.841667,99.075833,14655.208333,-1.884900,1994,1,5,1,52,1994-01-01,17.0
1,1994-01-02,1994-01-02,1,-8.704167,-12.445833,74.416667,28.387500,99.920000,15783.333333,-12.827363,1994,1,6,2,52,1994-01-02,17.0
2,1994-01-03,1994-01-03,2,-12.612500,-15.820833,77.250000,10.404167,100.057083,18992.708333,-17.143932,1994,1,0,3,1,1994-01-03,17.0
3,1994-01-04,1994-01-04,3,-9.833333,-13.570833,74.416667,24.879167,98.675417,19080.708333,-14.183286,1994,1,1,4,1,1994-01-04,17.0
4,1994-01-05,1994-01-05,4,-11.866667,-16.154167,70.791667,27.033333,99.352083,19183.208333,-16.407695,1994,1,2,5,1,1994-01-05,17.0


In [150]:
df_pre.head()

,Unnamed: 0,temp_min,temp_max,temp_mean,temp_median,temp_max_hour,temp_min_hour,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,dew_point_temp_max_hour,dew_point_temp_min_hour,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,rel_hum_max_hour,rel_hum_min_hour,visibility_min,visibility_max,visibility_mean,visibility_median,visibility_max_hour,visibility_min_hour,press_min,press_max,press_mean,press_median,press_max_hour,press_min_hour,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median,hmdxx_max_hour,hmdxx_min_hour,sun_set,daily_demand,day_of_week,week_of_year,year,day,month,demand_rolling_mean_3,demand_rolling_std_7,demand_lag_1,demand_lag_2,demand_lag_3,demand_lag_4,demand_lag_5,demand_lag_6,demand_lag_7,date
0,0,-1.8,2.8,0.575000,0.65,12.0,5.0,-4.8,1.1,-1.387500,-0.65,19.0,7.0,73.0,99.0,87.000000,86.5,14.0,9.0,0.8,40.2,17.841667,16.1,11.0,14.0,98.51,99.91,99.075833,98.840,0.0,16.0,-4.976663,0.686009,-1.884900,-1.513575,13.0,7.0,17.0,14152.0,5.0,52.0,1994,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01
1,1,-14.3,1.7,-8.704167,-9.90,0.0,23.0,-20.3,0.5,-12.445833,-13.90,0.0,23.0,60.0,92.0,74.416667,75.5,0.0,23.0,6.4,40.2,28.387500,25.0,9.0,6.0,98.77,100.51,99.920000,100.125,18.0,0.0,-19.165499,-0.338394,-12.827363,-14.393998,0.0,23.0,17.0,16527.0,6.0,52.0,1994,2,1,NaN,NaN,14152.0,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-02
2,2,-16.3,-10.3,-12.612500,-12.00,17.0,2.0,-23.2,-12.6,-15.820833,-15.00,17.0,2.0,55.0,91.0,77.250000,80.5,9.0,2.0,1.2,25.0,10.404167,8.0,0.0,9.0,99.41,100.47,100.057083,100.155,0.0,23.0,-21.317386,-14.555717,-17.143932,-16.423198,17.0,2.0,17.0,17550.0,0.0,1.0,1994,3,1,16076.333333,NaN,16527.0,14152.0,NaN,NaN,NaN,NaN,NaN,1994-01-03
3,3,-11.5,-7.4,-9.833333,-9.95,15.0,8.0,-15.0,-11.5,-13.570833,-13.70,13.0,0.0,57.0,84.0,74.416667,76.0,10.0,16.0,9.7,40.2,24.879167,24.1,13.0,10.0,98.27,99.34,98.675417,98.610,0.0,14.0,-15.874948,-11.706660,-14.183286,-14.310177,15.0,8.0,17.0,17395.0,1.0,1.0,1994,4,1,17157.333333,NaN,17550.0,16527.0,14152.0,NaN,NaN,NaN,NaN,1994-01-04
4,4,-17.5,-7.2,-11.866667,-11.75,5.0,22.0,-20.8,-10.7,-16.154167,-17.00,4.0,22.0,56.0,84.0,70.791667,74.0,4.0,14.0,1.0,40.2,27.033333,25.0,9.0,7.0,98.44,100.37,99.352083,99.375,22.0,3.0,-22.394182,-11.256395,-16.407695,-16.464188,5.0,22.0,17.0,18485.0,2.0,1.0,1994,5,1,17810.000000,NaN,17395.0,17550.0,16527.0,14152.0,NaN,NaN,NaN,1994-01-05


In [151]:
df_orig.drop(columns=['day_of_year', 'date_x', 'date_y', 'key_0', 'Unnamed: 0'], inplace=True)

In [152]:
df_orig.head()

,temp,dew_point_temp,rel_hum,visibility,press,daily_demand,hmdxx,year,month,day_of_week,week_of_year,sun_set
0,0.575000,-1.387500,87.000000,17.841667,99.075833,14655.208333,-1.884900,1994,1,5,52,17.0
1,-8.704167,-12.445833,74.416667,28.387500,99.920000,15783.333333,-12.827363,1994,1,6,52,17.0
2,-12.612500,-15.820833,77.250000,10.404167,100.057083,18992.708333,-17.143932,1994,1,0,1,17.0
3,-9.833333,-13.570833,74.416667,24.879167,98.675417,19080.708333,-14.183286,1994,1,1,1,17.0
4,-11.866667,-16.154167,70.791667,27.033333,99.352083,19183.208333,-16.407695,1994,1,2,1,17.0


In [153]:
df_pre.head()

,Unnamed: 0,temp_min,temp_max,temp_mean,temp_median,temp_max_hour,temp_min_hour,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,dew_point_temp_max_hour,dew_point_temp_min_hour,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,rel_hum_max_hour,rel_hum_min_hour,visibility_min,visibility_max,visibility_mean,visibility_median,visibility_max_hour,visibility_min_hour,press_min,press_max,press_mean,press_median,press_max_hour,press_min_hour,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median,hmdxx_max_hour,hmdxx_min_hour,sun_set,daily_demand,day_of_week,week_of_year,year,day,month,demand_rolling_mean_3,demand_rolling_std_7,demand_lag_1,demand_lag_2,demand_lag_3,demand_lag_4,demand_lag_5,demand_lag_6,demand_lag_7,date
0,0,-1.8,2.8,0.575000,0.65,12.0,5.0,-4.8,1.1,-1.387500,-0.65,19.0,7.0,73.0,99.0,87.000000,86.5,14.0,9.0,0.8,40.2,17.841667,16.1,11.0,14.0,98.51,99.91,99.075833,98.840,0.0,16.0,-4.976663,0.686009,-1.884900,-1.513575,13.0,7.0,17.0,14152.0,5.0,52.0,1994,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01
1,1,-14.3,1.7,-8.704167,-9.90,0.0,23.0,-20.3,0.5,-12.445833,-13.90,0.0,23.0,60.0,92.0,74.416667,75.5,0.0,23.0,6.4,40.2,28.387500,25.0,9.0,6.0,98.77,100.51,99.920000,100.125,18.0,0.0,-19.165499,-0.338394,-12.827363,-14.393998,0.0,23.0,17.0,16527.0,6.0,52.0,1994,2,1,NaN,NaN,14152.0,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-02
2,2,-16.3,-10.3,-12.612500,-12.00,17.0,2.0,-23.2,-12.6,-15.820833,-15.00,17.0,2.0,55.0,91.0,77.250000,80.5,9.0,2.0,1.2,25.0,10.404167,8.0,0.0,9.0,99.41,100.47,100.057083,100.155,0.0,23.0,-21.317386,-14.555717,-17.143932,-16.423198,17.0,2.0,17.0,17550.0,0.0,1.0,1994,3,1,16076.333333,NaN,16527.0,14152.0,NaN,NaN,NaN,NaN,NaN,1994-01-03
3,3,-11.5,-7.4,-9.833333,-9.95,15.0,8.0,-15.0,-11.5,-13.570833,-13.70,13.0,0.0,57.0,84.0,74.416667,76.0,10.0,16.0,9.7,40.2,24.879167,24.1,13.0,10.0,98.27,99.34,98.675417,98.610,0.0,14.0,-15.874948,-11.706660,-14.183286,-14.310177,15.0,8.0,17.0,17395.0,1.0,1.0,1994,4,1,17157.333333,NaN,17550.0,16527.0,14152.0,NaN,NaN,NaN,NaN,1994-01-04
4,4,-17.5,-7.2,-11.866667,-11.75,5.0,22.0,-20.8,-10.7,-16.154167,-17.00,4.0,22.0,56.0,84.0,70.791667,74.0,4.0,14.0,1.0,40.2,27.033333,25.0,9.0,7.0,98.44,100.37,99.352083,99.375,22.0,3.0,-22.394182,-11.256395,-16.407695,-16.464188,5.0,22.0,17.0,18485.0,2.0,1.0,1994,5,1,17810.000000,NaN,17395.0,17550.0,16527.0,14152.0,NaN,NaN,NaN,1994-01-05


In [154]:
df_pre.drop(columns=['date'], inplace=True)

In [155]:
df_orig.isnull().sum()

temp              0
dew_point_temp    0
rel_hum           0
visibility        0
press             0
daily_demand      0
hmdxx             0
year              0
month             0
day_of_week       0
week_of_year      0
sun_set           0
dtype: int64

In [156]:
df_orig.to_csv('/Users/yashwanthkaruparthi/Developer/energy_demand/research/data/9-basic-feat/feat-basic.csv')